In [2]:
import pandas as pd

# Load the datasets
updated_filtered_data = pd.read_csv('cleaned_filtered_merged_sensor_weather_data.csv')


In [3]:
def replace_consecutive_values(df, column, threshold=60):
    consecutive_count = 0
    current_value = None
    for i in range(len(df)):
        if df[column].iloc[i] == current_value:
            consecutive_count += 1
        else:
            if consecutive_count > threshold:
                df.loc[i - consecutive_count:i, column] = pd.NA
            current_value = df[column].iloc[i]
            consecutive_count = 1
    if consecutive_count > threshold:
        df.loc[len(df) - consecutive_count:len(df), column] = pd.NA
    return df

for col in ['Temperature', 'Humidity', 'Temperature inlet']:
    replace_consecutive_values(updated_filtered_data, col)

updated_filtered_data.to_csv('final_cleaned_filtered_sensor_weather_data.csv', index=False)


In [5]:
updated_filtered_data['Nedbør (12 t)'].fillna(0, inplace=True)
updated_filtered_data['Time'] = pd.to_datetime(updated_filtered_data['Time'])

cleaned_data_strict = updated_filtered_data[
    ~(
        (updated_filtered_data['Time'].dt.month >= 9) & 
        (updated_filtered_data['Time'].dt.month <= 12) & 
        (updated_filtered_data[['Temperature', 'Humidity', 'Temperature inlet']].isna().all(axis=1))
    )
]

In [6]:
cleaned_data_strict.to_csv('merged_data3.csv', index=False)

In [7]:
merged_data = pd.read_csv('merged_data5.csv')

In [8]:
merged_data['Dew Point'] = merged_data['Temperature'] - ((100 - merged_data['Humidity']) / 5)

In [9]:
merged_data.to_csv('merged_data6.csv')

In [10]:
import numpy as np

In [12]:
final_merged_data = merged_data

final_merged_data['Time'] = pd.to_datetime(final_merged_data['Time'])

final_merged_data['hour'] = final_merged_data['Time'].dt.hour
final_merged_data['day_of_year'] = final_merged_data['Time'].dt.dayofyear

# Compute sine and cosine for time of day
final_merged_data['time_sine'] = np.sin(2 * np.pi * final_merged_data['hour'] / 24)
final_merged_data['time_cosine'] = np.cos(2 * np.pi * final_merged_data['hour'] / 24)

# Compute sine and cosine for day of the year
final_merged_data['day_sine'] = np.sin(2 * np.pi * final_merged_data['day_of_year'] / 365)
final_merged_data['day_cosine'] = np.cos(2 * np.pi * final_merged_data['day_of_year'] / 365)

# Drop the intermediate 'hour' and 'day_of_year' columns
final_merged_data.drop(columns=['hour', 'day_of_year'], inplace=True)

final_merged_data.to_csv('merged_data7.csv')

In [16]:
# Ensure 'Time' is in datetime format
#final_merged_data['Time'] = pd.to_datetime(final_merged_data['Time'])

# Set 'Time' as the index
#final_merged_data.set_index('Time', inplace=True)

# Compute 3-hour rolling averages
final_merged_data['Temperature_3h_avg'] = final_merged_data['Temperature'].rolling(window='3H', min_periods=1).mean()
final_merged_data['Humidity_3h_avg'] = final_merged_data['Humidity'].rolling(window='3H', min_periods=1).mean()
final_merged_data['Temperature_inlet_3h_avg'] = final_merged_data['Temperature inlet'].rolling(window='3H', min_periods=1).mean()
final_merged_data['Dew_Point_3h_avg'] = final_merged_data['Dew Point'].rolling(window='3H', min_periods=1).mean()

# Compute 12-hour rolling averages
final_merged_data['Temperature_12h_avg'] = final_merged_data['Temperature'].rolling(window='12H', min_periods=1).mean()
final_merged_data['Humidity_12h_avg'] = final_merged_data['Humidity'].rolling(window='12H', min_periods=1).mean()
final_merged_data['Temperature_inlet_12h_avg'] = final_merged_data['Temperature inlet'].rolling(window='12H', min_periods=1).mean()
final_merged_data['Dew_Point_12h_avg'] = final_merged_data['Dew Point'].rolling(window='12H', min_periods=1).mean()

final_merged_data.to_csv('merged_data8.csv')


In [20]:
data = pd.read_csv('merged_data8.csv')
cast_error_points = pd.read_csv('cast_error_data_updated.csv')

cast_error_points['datostøpt'] = pd.to_datetime(cast_error_points['datostøpt'])
data['Time'] = pd.to_datetime(data['Time'])

# Resample the cast error data to the closest hour
cast_error_points['datostøpt'] = cast_error_points['datostøpt'].dt.round('H')

# Merge the resampled cast error data with the sensor and weather data
merged_cast_error_with_sensor = pd.merge(
    cast_error_points,
    data,
    left_on='datostøpt',
    right_on='Time',
    how='inner'
)

columns_to_drop = ['regnr', 'datoVrakReg', 'antall vrak', 'vrakårsak', 'godkjent', 'Time']
cleaned_data = merged_cast_error_with_sensor.drop(columns=columns_to_drop)

# Drop rows without any sensor data (NaN values in sensor data columns)
sensor_data_columns = [
    'Nedbør (12 t)', 'Lufttemperatur', 'Temperature', 'Humidity', 'Temperature inlet',
    'Dew Point', 'Temperature_3h_avg', 'Humidity_3h_avg', 'Temperature_inlet_3h_avg',
    'Dew_Point_3h_avg', 'Temperature_12h_avg', 'Humidity_12h_avg',
    'Temperature_inlet_12h_avg', 'Dew_Point_12h_avg'
]
cleaned_data = cleaned_data.dropna(subset=sensor_data_columns)

# Save the cleaned dataset to a CSV file
cleaned_output_file_path = 'merged_data9.csv'
cleaned_data.to_csv(cleaned_output_file_path, index=False)

'merged_data9.csv'